# Setup

In [ ]:
%%capture
!sudo add-apt-repository -y ppa:openjdk-r/ppa
!sudo apt-get purge openjdk-*
!sudo apt-get install openjdk-8-jdk
!sudo apt-get install xvfb xserver-xephyr vnc4server python-opengl ffmpeg
!pip3 install --upgrade minerl
!pip3 install pyvirtualdisplay
!pip3 install pytorch
!pip3 install -U colabgymrender
!pip3 install wandb
!pip install torch_tb_profiler
!pip install hydra-core
!pip install kornia
!pip install flatten-dict


In [ ]:
!pip3 install --upgrade minerl


# Download the data and code

In [ ]:
%env MINERL_DATA_ROOT=/content/data/
!python3 -m minerl.data.download --environment "MineRLBasaltFindCave-v0"
!python3 -m minerl.data.download --environment "MineRLBasaltBuildVillageHouse-v0"
!python3 -m minerl.data.download --environment "MineRLTreechop-v0"
!python3 -m minerl.data.download --environment "MineRLNavigateExtremeDense-v0"
!python3 -m minerl.data.download --environment "MineRLNavigateDense-v0"
!ls $MINERL_DATA_ROOT


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
%cd /content/gdrive/MyDrive/basalt-competition
!pwd
%env MINERL_DATA_ROOT=/content/data/
!ls $MINERL_DATA_ROOT


In [ ]:
import wandb
wandb.login()

# Curious IQ

In [ ]:
!python train_submission_code.py \
method=curious_iq \
project_name=test \
env=cave \
method.curiosity_pretraining_steps=100 \
method.curiosity_only_steps=20 \
method.curiosity_fade_out_steps=10 \
method.starting_steps=500 \
method.training_steps=200 \
checkpoint_frequency=100 \
method.batch_size=8 \
--debug-env --wandb-false

In [ ]:
!python train_submission_code.py \
method=curious_iq \
env=cave \
method.initial_curiosity_fraction=0.25 \
project_name=CuriousIQ

# IQ Learn SAC

In [ ]:
!python train_submission_code.py \
method=iqlearn_sac \
project_name=test \
env=navigate_extreme \
method.starting_steps=50 \
method.training_steps=200 \
checkpoint_frequency=100 \
method.batch_size=8 \
--debug-env --wandb-false

In [ ]:
!python train_submission_code.py \
method=iqlearn_sac \
env=navigate_extreme \
method.starting_steps=500 \
project_name=test

# IQ-Learn Online


In [ ]:
!python train_submission_code.py \
method=iqlearn_online \
env=cave \
checkpoint_frequency=100 \
method.starting_steps=100 \
method.training_steps=1000 \
eval_frequency=100 \
project_name=debug --debug-env

In [ ]:
!python train_submission_code.py \
method=iqlearn_online \
env=navigate \
project_name=test

# Obs space experiments

In [ ]:
!python bc_experiment.py

In [ ]:
!wandb sweep sweeps/model_size.yaml

In [ ]:
!wandb agent basalt/ObservationSpace/62wxr2vg

# Optimization

In [ ]:
!python -m cProfile -o output.pstats train_submission_code.py --profile --debug-env project_name=profile random_translate=false

In [ ]:
!pip install graphviz
!pip install gprof2dot
!gprof2dot -f pstats output.pstats | dot -Tpng -o output.png

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./logs